# **The Parity function**

## **1 : Libraries**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# this 'device' will be used for training our model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## **2 : Data**

In [2]:
def gen_bits(bit):
    x = np.random.random_integers(bit-1, size=np.random.randint(bit))
    t = torch.zeros(bit,1)
    for i in x:
        t[i][0] = 1
    parity = int(t.sum().item()%2 == 1)
    return t, parity

def gen_data(size, bit=64):
    data = []
    for i in range(size):
        data.append(gen_bits(bit))
    return data

In [3]:
data_2000 = gen_data(2000) #dataset for 2000 size
data_5000 = gen_data(5000) #dataset for 5000 size
data_test = gen_data(800) #test dataset for both 2000 and 5000 size data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. Please call randint(1, 63 + 1) instead
  


In [4]:
#Converting list to torch tensors
train_loader_2000 = torch.utils.data.DataLoader(data_2000, batch_size=32, shuffle=True)
train_loader_5000 = torch.utils.data.DataLoader(data_5000, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=800, shuffle=True)

## **3 : Fully Connected Neural Network for the Parity dataset**

### Helper functions for training and testing

In [5]:
# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

accuracy_list = []

criterion = nn.BCELoss()

# we pass a model object to this trainer, and it trains this model for one epoch
def train(epoch, model, print_set = 20):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # send to device
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data).squeeze()
        loss = criterion(output, target.type(torch.float))
        loss.backward()
        optimizer.step()

        pred = torch.round(output)
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    accuracy = 100. * correct / len(train_loader.dataset)
    print("Epoch:",epoch,"Training accuracy: ",accuracy,"%")
            
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        # send to device
        data, target = data.to(device), target.to(device)
        
        output = model(data).squeeze()
        test_loss += criterion(output, target.type(torch.float)).sum().item() # sum up batch loss
        pred = torch.round(output)
        # pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

### Defining the Fully Connected Network

In [6]:
class FC2Layer(nn.Module):
    def __init__(self, input_size, output_size):
        super(FC2Layer, self).__init__()
        self.input_size = input_size
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64,32), 
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(-1, self.input_size)
        return self.network(x)

### Train the Network for 2000 size data

In [7]:
input_size = 64
output_size = 1
print("Training on ", device)
model_fnn = FC2Layer(input_size, output_size)
model_fnn.to(device)
optimizer = optim.Adam(model_fnn.parameters())
print('Number of parameters: {}\n'.format(get_n_params(model_fnn)))

train_loader = train_loader_2000

for epoch in range(20):
    train(epoch, model_fnn)
    test(model_fnn)
    print('\n')

Training on  cpu
Number of parameters: 6785

Epoch: 0 Training accuracy:  48.45 %
Test set: Average loss: 0.0009, Accuracy: 422/800 (53%)


Epoch: 1 Training accuracy:  51.9 %
Test set: Average loss: 0.0009, Accuracy: 414/800 (52%)


Epoch: 2 Training accuracy:  51.85 %
Test set: Average loss: 0.0009, Accuracy: 414/800 (52%)


Epoch: 3 Training accuracy:  52.95 %
Test set: Average loss: 0.0009, Accuracy: 421/800 (53%)


Epoch: 4 Training accuracy:  57.25 %
Test set: Average loss: 0.0009, Accuracy: 421/800 (53%)


Epoch: 5 Training accuracy:  60.75 %
Test set: Average loss: 0.0009, Accuracy: 420/800 (52%)


Epoch: 6 Training accuracy:  63.15 %
Test set: Average loss: 0.0009, Accuracy: 421/800 (53%)


Epoch: 7 Training accuracy:  65.5 %
Test set: Average loss: 0.0009, Accuracy: 424/800 (53%)


Epoch: 8 Training accuracy:  69.75 %
Test set: Average loss: 0.0009, Accuracy: 424/800 (53%)


Epoch: 9 Training accuracy:  75.65 %
Test set: Average loss: 0.0010, Accuracy: 427/800 (53%)


Epoch: 

### Train the Network for 5000 size data

In [8]:
input_size = 64
output_size = 1
print("Training on ", device)
model_fnn = FC2Layer(input_size, output_size)
model_fnn.to(device)
optimizer = optim.Adam(model_fnn.parameters())
print('Number of parameters: {}\n'.format(get_n_params(model_fnn)))

train_loader = train_loader_5000

for epoch in range(20):
    train(epoch, model_fnn)
    test(model_fnn)
    print('\n')

Training on  cpu
Number of parameters: 6785

Epoch: 0 Training accuracy:  48.8 %
Test set: Average loss: 0.0009, Accuracy: 397/800 (50%)


Epoch: 1 Training accuracy:  51.38 %
Test set: Average loss: 0.0009, Accuracy: 404/800 (50%)


Epoch: 2 Training accuracy:  52.44 %
Test set: Average loss: 0.0009, Accuracy: 421/800 (53%)


Epoch: 3 Training accuracy:  54.06 %
Test set: Average loss: 0.0009, Accuracy: 386/800 (48%)


Epoch: 4 Training accuracy:  55.76 %
Test set: Average loss: 0.0009, Accuracy: 395/800 (49%)


Epoch: 5 Training accuracy:  58.08 %
Test set: Average loss: 0.0009, Accuracy: 385/800 (48%)


Epoch: 6 Training accuracy:  61.34 %
Test set: Average loss: 0.0009, Accuracy: 386/800 (48%)


Epoch: 7 Training accuracy:  64.3 %
Test set: Average loss: 0.0009, Accuracy: 404/800 (50%)


Epoch: 8 Training accuracy:  67.76 %
Test set: Average loss: 0.0010, Accuracy: 391/800 (49%)


Epoch: 9 Training accuracy:  70.54 %
Test set: Average loss: 0.0011, Accuracy: 396/800 (50%)


Epoch: 

We can see that using both dataset we are getting traninig good accuracy, but getting only 50% test accuracy.
We know there are $2^{64}$ possible combination of data points and we are only considering very few data points in our training.
As it is not possible to take majority of data points in training, we will use manually created DNN for this task.

## **4 : Manual Design of a Neural network for Parity dataset**

Design of the Neural Network:
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.80.5442&rep=rep1&type=pdf

In [9]:
class manual_nn():
  def __init__(self, bit):
    self.rows = bit + (bit//2)
    self.bit = bit
    self.W1 = self.initialize_W1()
    self.b1 = self.initialize_b1()
    self.W2 = self.initialize_W2()
    self.b2 = torch.tensor(-0.5)

  def forward(self, x, print = False):
    if print:
      print("x: ",x.shape)
    a1 = torch.matmul(self.W1, x) + self.b1
    if print:
      print("a1: ",a1.shape)
    h1 = self.steps(a1, self.rows)
    if print:
      print("h1: ",h1.shape)
    a2 = torch.matmul(self.W2, h1) + self.b2
    if print:
      print("a2: ",a2.shape)
    output = self.steps(a2, 1)
    if print:
      print("out: ",output.shape)
    return output.item()

  def initialize_W1(self):
    rows = self.rows
    cols = self.bit
    temp = torch.zeros(rows,cols)
    for i in range(cols):
      temp[i][i] = 1
    for i in range(cols, rows):
      for j in range(cols):
        temp[i][j] = 1
    return temp

  def initialize_b1(self):
    rows = self.rows
    temp = torch.zeros(rows,1)
    t = 0.5
    for i in range(self.bit, rows):
      temp[i][0] = t-2
      t -= 2 
    return temp

  def initialize_W2(self):
    rows = self.rows
    temp = torch.ones(1, rows)
    for i in range(self.bit, rows):
      temp[0][i] = -2
    return temp

  def steps(self, vect, size):
    temp = torch.zeros(size,1)
    for i in range(size):
      if vect[i][0]>0:
        temp[i][0] = 1
    return temp
  
  def predict(self,x):
    temp = torch.zeros(len(x))
    for i in range(len(x)):
      temp[i] = self.forward(x[i])
    return temp


In [10]:
print("Training on ", device)
model_manual = manual_nn(64)
for images,labels in test_loader:
  pred = model_manual.predict(images)
print("Test set: Accuracy {}%".format(int(sum(pred == labels)*100/len(labels))))

Training on  cpu
Test set: Accuracy 100%


### Testing on large data

In [11]:
data_test = gen_data(10000)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=800, shuffle=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. Please call randint(1, 63 + 1) instead
  


In [12]:
print("Training on ", device)
model_manual = manual_nn(64)
for images,labels in test_loader:
  pred = model_manual.predict(images)
print("Test set: Accuracy {}%".format(int(sum(pred == labels)*100/len(labels))))

Training on  cpu
Test set: Accuracy 100%


Even on larger test data manual network is giving 100% accuracy